# mAP computation demo
## Enable logging

In [1]:
import logging
logging.basicConfig(format='%(levelname)s:%(asctime)s:%(name)s:%(message)s')
logging.getLogger("copairs").setLevel(logging.INFO)

In [2]:
import pandas as pd
import numpy as np
from copairs.map import run_pipeline

## Utils functions

In [3]:
plate_col = 'Metadata_Plate'
well_col = 'Metadata_Well'
pert_col = 'Metadata_JCP2022'

def mock_plates(n_plates, n_wells, n_perts, seed=0):
    rng = np.random.default_rng(seed)
    n_tries = (n_plates * n_wells * n_perts) * 3
    n_feats = 3
    dframe = pd.DataFrame({
        plate_col: rng.choice([f'p{i}' for i in range(n_plates)], n_tries),
        well_col: rng.choice([f'w{i}' for i in range(n_wells)], n_tries),
        pert_col: rng.choice([f'c{i}' for i in range(n_perts)], n_tries)
    })

    dframe = dframe.drop_duplicates([plate_col, well_col])
    dframe = dframe.sort_values(by=[plate_col, well_col, pert_col])
    dframe = dframe.reset_index(drop=True)
    feats = rng.uniform(size=(dframe.shape[0], n_feats))
    return dframe, feats

## Create a mock dataset
Creating a mock dataset with 3 plates, 5 wells per each plate, and 4 different perturbations

In [4]:
# this is similar to Roshans's code
n_plates, n_wells, n_perts = 3, 5, 4
dframe, feats = mock_plates(n_plates, n_wells, n_perts)
dframe

,Metadata_Plate,Metadata_Well,Metadata_JCP2022
0,p0,w0,c0
1,p0,w1,c2
2,p0,w2,c2
3,p0,w3,c1
4,p0,w4,c0
5,p1,w0,c1
6,p1,w1,c2
7,p1,w2,c2
8,p1,w3,c3
9,p1,w4,c3


## Define params to compute map

In [5]:
pos_sameby = [pert_col]
pos_diffby = [plate_col, well_col]

neg_sameby = [plate_col]
neg_diffby = [pert_col]
null_size =10000

In [6]:
pos_sameby

['Metadata_JCP2022']

In [7]:
print("positive")
print(pos_sameby)
print(pos_diffby)

print("negative")
print(neg_sameby)
print(neg_diffby)

positive
['Metadata_JCP2022']
['Metadata_Plate', 'Metadata_Well']
negative
['Metadata_Plate']
['Metadata_JCP2022']


In [8]:
feats

array([[0.74438073, 0.85187591, 0.13893168],
       [0.70378577, 0.82110309, 0.98182832],
       [0.84379056, 0.42410649, 0.97968871],
       [0.9739844 , 0.50367698, 0.75344654],
       [0.91383767, 0.47614707, 0.86378624],
       [0.70156857, 0.29392426, 0.76765227],
       [0.57068479, 0.09384515, 0.39138043],
       [0.07374101, 0.47616696, 0.42853961],
       [0.42373744, 0.58630035, 0.12269066],
       [0.93376891, 0.68405045, 0.82378136],
       [0.89680123, 0.58332005, 0.04021822],
       [0.71148682, 0.56902585, 0.82595722],
       [0.53216047, 0.8132441 , 0.99701029],
       [0.35055481, 0.17102144, 0.3916748 ],
       [0.75304999, 0.43922893, 0.58838011]])

In [9]:
result = run_pipeline(dframe, feats,
                      pos_sameby, pos_diffby,
                      neg_sameby, neg_diffby,
                      null_size
                     )

INFO:2023-12-14 22:37:46,247:copairs:Indexing metadata...
INFO:2023-12-14 22:37:46,255:copairs:Finding positive pairs...
INFO:2023-12-14 22:37:46,257:copairs:Finding negative pairs...


finding pos pairs
from samby['all']
{'c1': [(3, 5)], 'c2': [(1, 13), (1, 14), (1, 7), (2, 13), (2, 14), (2, 6), (6, 13), (6, 14), (7, 13), (7, 14)], 'c3': [(8, 10), (8, 11), (8, 12), (9, 10), (9, 11), (9, 12)]}
{'c1': [(3, 5)], 'c2': [(1, 13), (1, 14), (1, 7), (2, 13), (2, 14), (2, 6), (6, 13), (6, 14), (7, 13), (7, 14)], 'c3': [(8, 10), (8, 11), (8, 12), (9, 10), (9, 11), (9, 12)]}
finding neg pairs
from samby['all']
{'p0': [(0, 1), (0, 2), (0, 3), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)], 'p1': [(5, 8), (5, 9), (5, 6), (5, 7), (6, 8), (6, 9), (7, 8), (7, 9)], 'p2': [(10, 13), (10, 14), (11, 13), (11, 14), (12, 13), (12, 14)]}
{'p0': [(0, 1), (0, 2), (0, 3), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)], 'p1': [(5, 8), (5, 9), (5, 6), (5, 7), (6, 8), (6, 9), (7, 8), (7, 9)], 'p2': [(10, 13), (10, 14), (11, 13), (11, 14), (12, 13), (12, 14)]}


In [7]:
result

,Metadata_Plate,Metadata_Well,Metadata_JCP2022,p_value,null_dists,average_precision
0,p0,w0,c0,0.000100,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN
1,p0,w1,c2,0.556044,"[0.7222222222222222, 0.41111111111111115, 0.46...",0.588889
2,p0,w2,c2,0.357764,"[0.8333333333333334, 1.0, 0.6666666666666666, ...",0.700000
3,p0,w3,c1,0.198180,"[0.25, 0.5, 1.0, 1.0, 0.25, 0.5, 0.5, 0.2, 1.0...",0.500000
4,p0,w4,c0,0.000100,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN
5,p1,w0,c1,0.000100,"[1.0, 0.2, 0.3333333333333333, 0.2, 1.0, 1.0, ...",1.000000
6,p1,w1,c2,0.446855,"[0.5888888888888889, 0.7000000000000001, 0.866...",0.638889
7,p1,w2,c2,0.255174,"[0.8333333333333334, 1.0, 0.6666666666666666, ...",0.755556
8,p1,w3,c3,0.000100,"[0.8333333333333334, 1.0, 0.6666666666666666, ...",1.000000
9,p1,w4,c3,0.255174,"[0.8333333333333334, 1.0, 0.6666666666666666, ...",0.755556


<div class="alert alert-warning">
    WARNING: <code>NaN</code> average precision mean such rows do not have any valid match.
</div>

# Aggregating p-values

For every sample in the dataset you have one associated `average_precision` score and the corresponding `p_value`. You may want to aggregate those results while correcting for false discoveries.

## Create a larger dataset

In [11]:
n_plates, n_wells, n_perts = 4, 384, 50
pos_sameby = [pert_col]
pos_diffby = [plate_col, well_col]

neg_sameby = [plate_col]
neg_diffby = [pert_col]
null_size =10000

dframe, feats = mock_plates(n_plates, n_wells, n_perts)
result = run_pipeline(dframe, feats,
                      pos_sameby, pos_diffby,
                      neg_sameby, neg_diffby,
                      null_size
                     )

INFO:2023-12-05 22:27:33,244:copairs:Indexing metadata...
INFO:2023-12-05 22:27:33,251:copairs:Finding positive pairs...
INFO:2023-12-05 22:27:33,268:copairs:Finding negative pairs...
INFO:2023-12-05 22:27:33,432:copairs:Computing positive similarities...


  0%|          | 0/1 [00:00<?, ?it/s]

INFO:2023-12-05 22:27:33,442:copairs:Computing negative similarities...


  0%|          | 0/15 [00:00<?, ?it/s]

INFO:2023-12-05 22:27:33,465:copairs:Building rank lists...
INFO:2023-12-05 22:27:33,577:copairs:Computing average precision...
INFO:2023-12-05 22:27:33,595:copairs:Computing p-values...


  0%|          | 0/150 [00:00<?, ?it/s]

INFO:2023-12-05 22:27:37,987:copairs:Creating result DataFrame...
INFO:2023-12-05 22:27:37,990:copairs:Finished.


## Aggregate average precision and p-values

In [12]:
from copairs.map import aggregate
agg_result = aggregate(result, sameby=pos_sameby, threshold=0.05)
agg_result

,Metadata_JCP2022,mean_average_precision,nlog10pvalue,q_value,nlog10qvalue,above_p_threshold,above_q_threshold
0,c0,0.053035,0.318423,0.589071,0.229833,False,False
1,c1,0.072934,0.627046,0.589071,0.229833,False,False
2,c10,0.052094,0.548858,0.589071,0.229833,False,False
3,c11,0.077090,0.736562,0.589071,0.229833,False,False
4,c12,0.062313,0.392067,0.589071,0.229833,False,False
5,c13,0.061869,0.468056,0.589071,0.229833,False,False
6,c14,0.077471,0.479712,0.589071,0.229833,False,False
7,c15,0.053557,0.429338,0.589071,0.229833,False,False
8,c16,0.056858,0.204951,0.623806,0.204951,False,False
9,c17,0.072333,0.244594,0.593109,0.226865,False,False


Features are sampled from uniform distribution, thus it is expected that none of the perturbations surpass the significance `threshold=0.05`:

In [10]:
agg_result[['above_p_threshold', 'above_q_threshold']].value_counts()

above_p_threshold  above_q_threshold
False              False                50
Name: count, dtype: int64

## Plotting mAP vs p-values
The following plot requires `plotly` package for visualization. You can install it with:
```bash
$ pip install plotly
```

In [11]:
import plotly.express as px

p_curve = px.scatter(agg_result,
           x='average_precision',
           y='nlog10pvalue',
           hover_name=pert_col
          ).add_hline(y=-np.log10(0.05))
q_curve = px.scatter(agg_result,
           x='average_precision',
           y='nlog10qvalue',
           hover_name=pert_col
          ).add_hline(y=-np.log10(0.05))

In [12]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)

# Add the individual figures to the subplot
fig.add_trace(p_curve.data[0], row=1, col=1)
fig.add_trace(q_curve.data[0], row=1, col=2)

# Update the subplot layout
fig.update_layout(height=500, width=1000)

# Display the subplot
fig.add_hline(y=-np.log10(0.05)).show()